In [1]:
!pip install optuna
import optuna
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.7 MB/s eta 0:00:00


In [2]:
# 테이터 확인
df_train = pd.read_csv('/content/drive/MyDrive/DACON/채무 불이행 여부 예측 해커톤: 불이행의 징후를 찾아라!/train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  object 
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  object 
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  object 
 12  대출 상환 기간           10000 non-null  object 
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

In [3]:
# 결측치가 없음을 확인
# object 데이터 LabelEncoding 변환

column_list = list(df_train.columns)
column_list = column_list[1:2] + column_list[3:4] + column_list[11:13]

encoder = LabelEncoder()
for i in column_list:
  encoder.fit(df_train[i])
  df_train[i] = encoder.transform(df_train[i])

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  int64  
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  int64  
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  int64  
 12  대출 상환 기간           10000 non-null  int64  
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

In [4]:
# UID열 제거
df_train.drop(columns = ['UID'], inplace = True)

In [7]:
# Optuna를 사용하기 위한 모델 정의
def prediction_model(trial):
  x = df_train.drop(columns = ['채무 불이행 여부'])
  y = df_train['채무 불이행 여부']

  # 데이터 분할
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

  # optuna에 사용할 하이퍼파라미터 설정
  params = {
      'C':trial.suggest_float('C', 1e-5, 1e2, log = True),
      'penalty':'l2',
      'solver':'lbfgs',
      'max_iter':trial.suggest_int('max_iter', 100, 5000),
      }

  # 로지스틱 회귀모델 객체 생성
  log_regressor = LogisticRegression(**params)
  log_regressor.fit(x_train, y_train)

  # 예측값 도출
  y_pred = log_regressor.predict(x_test)
  ra_score = roc_auc_score(y_test, y_pred)

  return ra_score

In [8]:
# prediction_model의 최적 하이퍼파라미터 찾기
study = optuna.create_study(direction = 'maximize')
study.optimize(prediction_model, n_trials = 300)

optimum_params = study.best_params
print(optimum_params)

[I 2025-03-17 14:46:29,000] A new study created in memory with name: no-name-0f3b2fe0-0679-4532-9668-81f070c14e16
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2025-03-17 14:46:31,647] Trial 0 finished with value: 0.5587638873907884 and parameters: {'C': 1.4116520850239835, 'max_iter': 3249}. Best is trial 0 with value: 0.5587638873907884.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase

{'C': 60.85912377036622, 'max_iter': 3658}


In [22]:
# test 데이터 불러오기
df_test = pd.read_csv('/content/drive/MyDrive/DACON/채무 불이행 여부 예측 해커톤: 불이행의 징후를 찾아라!/test.csv')

# test 데이터 확인
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                2062 non-null   object 
 1   주거 형태              2062 non-null   object 
 2   연간 소득              2062 non-null   float64
 3   현재 직장 근속 연수        2062 non-null   object 
 4   체납 세금 압류 횟수        2062 non-null   float64
 5   개설된 신용계좌 수         2062 non-null   int64  
 6   신용 거래 연수           2062 non-null   float64
 7   최대 신용한도            2062 non-null   float64
 8   신용 문제 발생 횟수        2062 non-null   int64  
 9   마지막 연체 이후 경과 개월 수  2062 non-null   int64  
 10  개인 파산 횟수           2062 non-null   int64  
 11  대출 목적              2062 non-null   object 
 12  대출 상환 기간           2062 non-null   object 
 13  현재 대출 잔액           2062 non-null   float64
 14  현재 미상환 신용액         2062 non-null   float64
 15  월 상환 부채액           2062 non-null   float64
 16  신용 점수              2062 

In [23]:
# train 데이터 중 object 데이터 LabelEncoding 변환

test_column_list = list(df_test.columns)
test_column_list = test_column_list[1:2] + test_column_list[3:4] + test_column_list[11:13]

encoder = LabelEncoder()
for i in test_column_list:
  encoder.fit(df_test[i])
  df_test[i] = encoder.transform(df_test[i])

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                2062 non-null   object 
 1   주거 형태              2062 non-null   int64  
 2   연간 소득              2062 non-null   float64
 3   현재 직장 근속 연수        2062 non-null   int64  
 4   체납 세금 압류 횟수        2062 non-null   float64
 5   개설된 신용계좌 수         2062 non-null   int64  
 6   신용 거래 연수           2062 non-null   float64
 7   최대 신용한도            2062 non-null   float64
 8   신용 문제 발생 횟수        2062 non-null   int64  
 9   마지막 연체 이후 경과 개월 수  2062 non-null   int64  
 10  개인 파산 횟수           2062 non-null   int64  
 11  대출 목적              2062 non-null   int64  
 12  대출 상환 기간           2062 non-null   int64  
 13  현재 대출 잔액           2062 non-null   float64
 14  현재 미상환 신용액         2062 non-null   float64
 15  월 상환 부채액           2062 non-null   float64
 16  신용 점수              2062 

In [27]:
# 최적 하이퍼파라미터를 이용한 객체 생성
log_regressor = LogisticRegression(**optimum_params)

# 먼저 train 데이터를 통해 모델학습
x = df_train.drop(columns = ['채무 불이행 여부'])
y = df_train['채무 불이행 여부']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

log_regressor.fit(x_train, y_train)

# 최적 하이퍼파라미터를 이용한 학습모델에 test 데이터를 반영하여 채무 불이행 여부 도출
x = df_test.drop(columns = ['UID'])
y_default = log_regressor.predict(x)

df_trial = pd.DataFrame({'UID':df_test['UID'], '채무 불이행 확률':y_default})
df_trial

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0


In [30]:
df_trial.to_csv('/content/drive/MyDrive/DACON/채무 불이행 여부 예측 해커톤: 불이행의 징후를 찾아라!/submission_0318.csv', index = False)